# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  {'type': 'blog post',
   'url': 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'Facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'},
  {'type': 'external company',
   'url': 'https://neb

In [9]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [ ]:
select_relevant_links("https://edwarddonner.com")

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [11]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [12]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/HunyuanImage-3.0
Updated
about 3 hours ago
•
825
•
714
deepseek-ai/DeepSeek-V3.2-Exp
Updated
3 days ago
•
10.7k
•
470
tencent/Hunyuan3D-Part
Updated
4 days ago
•
2.57k
•
461
zai-org/GLM-4.6
Updated
2 days ago
•
9.72k
•
298
inclusionAI/Ring-1T-preview
Updated
1 day ago
•
818
•
208
Browse 1M+ models
Spaces
Running
1.29k
1.29k
Wan2.2 Animate
👁
Wan2.2 Animate
Running
14.5k
14.5k
DeepSite v3
🐳
Generate any application with DeepSeek
Running
on
Zero
MCP
1.46k
1.46k
Wan2.2 14B Fast
🎥
generate a video from an image with a text prompt


In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


"\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/HunyuanImage-3.0\nUpdated\nabout 4 hours ago\n•\n825\n•\n714\ndeepseek-ai/DeepSeek-V3.2-Exp\nUpdated\n3 days ago\n•\n10.7k\n•\n470\ntencent/Hunyuan3D-Part\nUpdated\n4 days ago\n•\n2.57k\n•\n461\nzai-org/GLM-4.6\nUpdated\n2 days ago\n•\n9.72k\n•\n298\ninclusionAI/Ring-1T-preview\nUpdated\n1 day ago\n•\n818\n•\n208\nBrowse 1M+ models\nSpaces\nRunning\n1.29k\n1.29k\nWan2.2 Animate\n👁\n

In [16]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [17]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the premier collaboration platform for the machine learning (ML) community, dedicated to building the future of artificial intelligence through openness and shared knowledge. It serves as a vibrant hub where ML engineers, scientists, and enthusiasts from across the globe can share, discover, and experiment with cutting-edge open-source machine learning models, datasets, and applications.

At its core, Hugging Face empowers the next generation of AI innovators to learn, collaborate, and contribute to an open and ethical AI future.

---

## What We Offer

- **Model Hub**: Access and browse over **1 million pre-trained models** covering all modalities including text, image, video, audio, and 3D.
- **Datasets**: Explore and use more than **250,000 datasets** curated for various machine learning tasks.
- **Spaces**: Deploy and share interactive ML applications and demos; over **400,000 apps** available and growing.
- **Compute Solutions**: Scalable paid compute services including optimized inference endpoints and GPU support starting at $0.60/hour.
- **Enterprise Solutions**: Customized offerings tailored to accelerate business adoption of machine learning technologies.
- **Open Source Stack**: Leverage the extensive Hugging Face open-source libraries to move faster in development.

---

## Our Community

Hugging Face is much more than a platform — it is a thriving, fast-growing community of machine learning practitioners who collectively build, maintain, and democratize AI resources. Collaborators range from individual researchers to top organizations, contributing models like:

- Tencent’s HunyuanImage series
- deepseek-ai’s DeepSeek models
- NVIDIA and Facebook’s datasets and models

Community members benefit from the ability to:

- Host and collaborate on unlimited public projects
- Share their work globally to build a professional ML portfolio
- Collaborate using state-of-the-art open-source tools  
 
---

## Company Culture

- **Collaborative:** Empowering users to co-create and improve AI together.
- **Open & Ethical:** Committed to building an open, transparent, and socially responsible AI ecosystem.
- **Innovative:** Constantly advancing the frontier in machine learning through community-driven innovation.
- **Inclusive:** Bringing together diverse contributors and users worldwide.

---

## Careers at Hugging Face

Join a team on the front lines of democratizing AI and machine learning! Hugging Face is looking for passionate engineers, researchers, data scientists, and product experts who want to make a global impact by enabling accessible, ethical, and collaborative AI development.

By working here, you'll:

- Collaborate with world-class ML experts
- Shape the future of open-source AI tools
- Impact a vibrant global community of hundreds of thousands of users  
- Thrive in an open culture that values creativity, diversity, and ethical AI  

Visit [huggingface.co/careers](https://huggingface.co/careers) to explore current job openings.

---

## Why Choose Hugging Face?

- The **largest and most diverse** collection of open-source machine learning resources.
- A **community-first platform** that fosters collaboration and rapid innovation.
- Flexible solutions for individuals, communities, startups, and enterprises.
- Trusted by industry leaders and researchers worldwide.

---

## Get Started Today

- Explore AI apps and models at [huggingface.co](https://huggingface.co)
- Join the community, build your portfolio, and accelerate your AI journey.
- Scale your projects with our pay-as-you-go compute resources.

**Hugging Face** — The AI Community Building the Future.

---

# Contact & Links

- Website: https://huggingface.co
- Community: https://huggingface.co/community
- Documentation: https://huggingface.co/docs
- Careers: https://huggingface.co/careers

---

*Hugging Face is your home for collaboration, innovation, and advancing machine learning together.*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


# Hugging Face: The AI Community Building the Future

---

## About Hugging Face

Hugging Face is a vibrant and fast-growing AI and machine learning community dedicated to advancing the future of open and ethical AI. It serves as a collaboration platform where machine learning engineers, scientists, and enthusiasts from around the world create, discover, and share machine learning models, datasets, and applications.

The Hugging Face Hub is the central place for hosting and collaborating on over **1 million models**, **250k+ datasets**, and **400k+ applications** spanning multiple modalities including text, images, video, audio, and 3D.

---

## What Hugging Face Offers

- **Models:** Browse and explore a vast collection of AI models, updated frequently by a diverse community. Popular trending models include image generators, video editors, and language models.
  
- **Datasets:** Access a rich catalog of datasets ranging from natural language prompts to persona datasets and beyond, enabling rapid model training and experimentation.
  
- **Spaces:** Deploy, share, and run AI applications on Hugging Face's platform, including innovative tools such as video generation from images, AI toolkits, and interactive applications.

- **Compute:** Easily accelerate your machine learning workloads with optimized inference endpoints and GPU-backed applications starting as low as $0.60/hour.

- **Enterprise Solutions:** Hugging Face provides tailored enterprise offerings to help organizations harness AI technology securely and effectively at scale.

---

## Company Culture

Hugging Face fosters an **open, collaborative, and ethical AI community** — empowering developers and researchers to work together transparently and inclusively. The culture emphasizes:

- **Community-driven innovation:** Encouraging open-source contributions and sharing knowledge.
- **Ethical AI development:** Committed to building responsible AI systems for the benefit of all.
- **Diversity and inclusion:** Supporting a broad and global community in building AI tools and resources.
- **Learning and growth:** Offering a platform for users to build their machine learning portfolios and grow their expertise.

---

## Customers & Community

Hugging Face serves a global community that includes:

- Machine learning researchers and practitioners
- Developers building innovative AI applications
- Enterprises seeking scalable AI solutions
- AI enthusiasts and learners who want to experiment and grow their skills

With over a million models and hundreds of thousands of datasets and applications, the platform connects users with cutting-edge AI technologies from companies like Tencent, NVIDIA, Facebook, and OpenAI — showcasing the power of open collaboration.

---

## Careers at Hugging Face

Interested in joining Hugging Face? The company values talented individuals passionate about AI, open-source technologies, and community building. Opportunities include roles in:

- Machine learning research & engineering
- Software development
- Cloud infrastructure and compute optimization
- Community management and developer relations
- Enterprise solutions & sales

At Hugging Face, you will work at the forefront of AI innovation while contributing to a global, open, and ethical AI movement.

---

## Get Involved

- **Explore AI models and datasets:** Visit [huggingface.co](https://huggingface.co) to browse millions of resources.
- **Build and share your own models and apps:** Create your portfolio in an open, collaborative environment.
- **Accelerate your projects:** Utilize Hugging Face’s Compute and Enterprise services.
- **Join the community:** Engage with thousands of practitioners working toward the future of AI.

---

### Contact & Follow

- Website: [https://huggingface.co](https://huggingface.co)
- Join the community to learn, collaborate, and innovate in AI!

---

*Hugging Face — Empowering the next generation of machine learning for a better, open future.*

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 14 relevant links


# Welcome to Hugging Face – The AI Community Building the Future!

---

## Who Are We?  
Imagine a place where machine learning wizards, data sorcerers, and AI alchemists gather to share their spells — uh, models — datasets, and apps. That’s Hugging Face! We’re *the* platform where the AI community collaborates, creates, and sometimes even has a little fun while building the future.

Our motto? **"Keep it open. Keep it ethical. Keep it hugging."** 💛

---

## What’s Cooking in the AI Kitchen?

- **1 Million+ Models** — From image generators to language wizards, our treasure trove of open-source ML models grows faster than you can say "neural network."  
- **250,000+ Datasets** — Feeding AI brains with everything from chat prompts to persona profiles. Hungry for data? Dig in!  
- **400,000+ Applications & Spaces** — Launch apps, share your ML portfolio, or just show off cool demos that make your friends say, “Whoa, AI can do that?”  
- **Multimodal Madness** — Text, image, video, audio, even 3D...if AI had a Swiss Army knife, we’d be it.  

---

## Customers & Community  
Whether you’re a student trying to get your AI feet wet, a startup looking to scale your genius, or an enterprise aiming to deploy heavy-duty models in the real world, Hugging Face has your back.

With the fastest growing community of *machine learning enthusiasts* and the support of some seriously big names and organizations, here’s a place where:

- **Freelancers** can build a portfolio and get noticed.  
- **Researchers** can push boundaries openly and ethically.  
- **Businesses** can accelerate AI adoption with our paid Compute and Enterprise suites.  

Join 1.29k+ Spaces and thousands more running models that power everything from video generation to AI-powered image editing.

---

## Culture & Career – Geek Out with Us!  
We believe collaboration beats isolation every day. Our culture?

- Open source at heart ❤️  
- Ethical AI advocates  
- Casual tea-drinkers and serious problem solvers  
- Always learning, always sharing, always growing  

Want to build machine learning tools that millions will use? Hugging Face is where your skills meet endless possibilities. From ML engineers to community managers, our doors are wide open (virtual hugs included).

---

## Speed Up Your AI Journey  
No need to code in the dark alone or fight for GPU time — deploy models and apps with a few clicks on optimized inference endpoints, starting at just $0.60/hour for GPU!

Whether you want to host that killer new model or just tweak an existing one, we give you the tools and community support to **move faster, build smarter, and hug tighter**.

---

## Quick Hugging Face Facts  
- **Founded:** Around the corner from the future  
- **Colors:** Bright yellow (#FFD21E), orange (#FF9D00), and sleek gray (#6B7280) — because AI should be as vibrant as its ideas!  
- **Mascot:** Friendly face with a warm smile (because AIs could learn a thing or two about friendliness here)  

---

## Ready to Join the AI Hug Circle?  

Sign up, share your work, explore millions of models and datasets, and get your AI career (or project!) hugging new heights.

[Explore AI Apps](#) | [Browse 1M+ Models](#) | [Sign Up & Join The Fun](#)

---

*Hugging Face — where the future of AI isn’t just created; it’s hugged into existence.* 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>